In [7]:
import duckdb
import openai
from dotenv import load_dotenv

In [8]:
con = duckdb.connect(database=':memory:')
df = con.execute("SELECT distinct description FROM read_csv('../data/raw/data.csv', ignore_errors=True)").fetchdf()

In [4]:
load_dotenv()
client = openai.OpenAI()

In [12]:
def get_embedding(text: str | None, model="text-embedding-3-small") -> list:
    if text is None:
        return []
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [13]:
df["ada_embedding"] = df["Description"].apply(lambda x: get_embedding(x))

In [15]:
df.to_csv("../data/raw/descriptions_with_embeddings.csv", index=False)

In [36]:
df_complete = con.execute("""
                  SELECT 
                    * 
                  FROM 
                    read_csv('../data/raw/data.csv', ignore_errors=True)
                  """).fetchdf()

In [37]:
df_merged = df_complete.merge(df, on="Description", how="left")

In [40]:
df_merged.to_csv("../data/transformed/data_with_embeddings.csv", index=False)